In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import xlwt

# 创建dataframe
df1 = pd.DataFrame(columns=['路线', '方向', '运行时间', '参考票价', '公交公司', '线路类型', '途径站点', '备注'])
df2 = pd.DataFrame(columns=['路线', '方向', '运行时间', '参考票价', '公交公司', '线路类型', '途径站点', '备注'])

In [2]:
def get_info1(url):
    print('正在爬取：', url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers = headers)
    response.encoding = 'UTF-8'
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 获取线路名信息
    line =[]
    linelist = soup.select('[class="name"]')
    for line_li in linelist:
        line.append(line_li.text)

    # 获取站点信息
    station = re.findall('<a href="/z_[0-9a-z]+" aria-label=".+?">(.+?)</a>', html, re.S)

    # 获取运行时间、参考票价、公交公司、备注信息
    info = []
    infolist = soup.select('[class="info"]')
    for info_li in infolist:
        info.append(info_li.text)
    if len(info) == 0:
        return
    info = info[0]
    index0 = info.index(']')
    index1 = info.index('参考')
    index2 = info.index('公交公司')
    index3 = info.index('最后')
    time = info[index0+1:index1].replace('运行时间：', '')
    price = info[index1:index2].replace('参考票价：', '')
    firm = info[index2:index3].replace('公交公司：', '')
    note = []
    notelist = soup.select('[class="change-info mb20"]')
    for note_li in notelist:
        note.append(note_li.text)
    if len(note) == 0:
        note = None
    else:
        note = note[0]
    # 获取线路类型信息
    line_type = re.findall('<a href="/line[0-9]" class="category" aria-label=".+?">(.+?)</a>', html, re.S)
    if len(line_type) == 0:
        return
    line_type = line_type[0]

     # 处理单线、环线、运行时间信息在网页位置不一致等情况
    if len(line) == 1:
        direction = station[0]+'-'+station[-1]
        line = line[0]
        df1.loc[len(df1.index)] = [line, direction, time, price, firm, line_type, station, note]
    elif len(line) > 1:
        index4 = -1
        line = line[0]
        for i in range(0, len(station)-1):
            if station[i] == station[i+1]:
                index4 = i+1
                break
        station1 = station[:i+1]
        station2 = station[i+1:]
        direction1 = station1[0] + '-' + station1[-1]
        direction2 = station1[-1] + '-' + station2[-1]
        if time.find('|') == -1:
            df1.loc[len(df1.index)] = [line, direction1, time, price, firm, line_type, station1, note]
            df1.loc[len(df1.index)] = [line, direction2, time, price, firm, line_type, station2, note]
        else:
            time = time.split('|')
            df1.loc[len(df1.index)] = [line, direction1, time[0], price, firm, line_type, station1, note]
            df1.loc[len(df1.index)] = [line, direction2, time[1], price, firm, line_type, station2, note]
    else:
        return

In [3]:
def get_info2(url):
    print('正在爬取：', url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers = headers)
    response.encoding = 'UTF-8'
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # 获取线路名信息
    line =[]
    linelist = soup.select('[class="name"]')
    for line_li in linelist:
        line.append(line_li.text)
        
    # 获取站点信息
    station = re.findall('<a href="/z_[0-9a-z]+" aria-label=".+?">(.+?)</a>', html, re.S)

    # 获取运行时间、参考票价、公交公司、备注信息
    info = []
    infolist = soup.select('[class="info"]')
    for info_li in infolist:
        info.append(info_li.text)
    if len(info) == 0:
        time = price = firm = None
    else:
        info = info[0]
        index0 = info.index(']')
        index1 = info.index('参考')
        index2 = info.index('公交公司')
        index3 = info.index('最后')
        time = info[index0+1:index1].replace('运行时间：', '')
        price = info[index1:index2].replace('参考票价：', '')
        firm = info[index2:index3].replace('公交公司：', '')
    note = []
    notelist = soup.select('[class="change-info mb20"]')
    for note_li in notelist:
        note.append(note_li.text)
    if len(note) == 0:
        note = None
    else:
        note = note[0]
        
    # 获取线路类型信息
    line_type = re.findall('<a href="/line[0-9]" class="category" aria-label=".+?">(.+?)</a>', html, re.S)
    if len(line_type) == 0:
        line_type = None
    else:
        line_type = line_type[0]

     # 处理单线、环线、运行时间信息在网页位置不一致等情况
    if len(line) == 1:
        direction = station[0]+'-'+station[-1]
        line = line[0]
        df2.loc[len(df2.index)] = [line, direction, time, price, firm, line_type, station, note]
    elif len(line) > 1:
        index4 = -1
        line = line[0]
        for i in range(0, len(station)-1):
            if station[i] == station[i+1]:
                index4 = i+1
                break
        station1 = station[:i+1]
        station2 = station[i+1:]
        direction1 = station1[0] + '-' + station1[-1]
        direction2 = station1[-1] + '-' + station2[-1]
        if time.find('|') == -1:
            df2.loc[len(df2.index)] = [line, direction1, time, price, firm, line_type, station1, note]
            df2.loc[len(df2.index)] = [line, direction1, time, price, firm, line_type, station2, note]
        else:
            time = time.split('|')
            df2.loc[len(df2.index)] = [line, direction1, time[0], price, firm, line_type, station1, note]
            df2.loc[len(df2.index)] = [line, direction2, time[1], price, firm, line_type, station2, note]
    else:
        return

In [4]:
# 获取所有公交线路详细信息页面的url
li = [1, 2, 3, 4, 5, 6, 7, 8, 9, 'B', 'C', 'D', 'G', 'H', 'K', 'M', 'T', 'X', 'Y', 'Z']
list_href = []
for i in li:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }
    head_url = 'https://xiamen.8684.cn/list{}'.format(i)
    response = requests.get(head_url, headers = headers)
    response.encoding = 'UTF-8'
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    urls = soup.select("[class='list clearfix']")
    head_url = head_url[0:-6]
    for url in urls:
        url_li = url.select("a")
        for j in url_li:
            url_href = j.get("href")
            list_href.append(head_url + url_href)
            out_url = list(set(list_href))

In [5]:
for i in out_url:
    get_info1(i)

正在爬取： https://xiamen.8684.cn/x_eb7745fd
正在爬取： https://xiamen.8684.cn/x_a4064afc
正在爬取： https://xiamen.8684.cn/x_bfe4a3eb
正在爬取： https://xiamen.8684.cn/x_7db14486
正在爬取： https://xiamen.8684.cn/x_4afa9e0d
正在爬取： https://xiamen.8684.cn/x_32d0d5d7
正在爬取： https://xiamen.8684.cn/x_a6a52ff7
正在爬取： https://xiamen.8684.cn/x_90e2ca26
正在爬取： https://xiamen.8684.cn/x_3ee317fa
正在爬取： https://xiamen.8684.cn/x_fec27493
正在爬取： https://xiamen.8684.cn/x_a3fc848f
正在爬取： https://xiamen.8684.cn/x_4542b81e
正在爬取： https://xiamen.8684.cn/x_92abb355
正在爬取： https://xiamen.8684.cn/x_dad74d6b
正在爬取： https://xiamen.8684.cn/x_a14b696b
正在爬取： https://xiamen.8684.cn/x_497aa162
正在爬取： https://xiamen.8684.cn/x_450c98e4
正在爬取： https://xiamen.8684.cn/x_bb5dd285
正在爬取： https://xiamen.8684.cn/x_7f2bbd44
正在爬取： https://xiamen.8684.cn/x_4f82091d
正在爬取： https://xiamen.8684.cn/x_36fc37ab
正在爬取： https://xiamen.8684.cn/x_ac5cf874
正在爬取： https://xiamen.8684.cn/x_5b9593cd
正在爬取： https://xiamen.8684.cn/x_6674c2bb
正在爬取： https://xiamen.8684.cn/x_e870d975


正在爬取： https://xiamen.8684.cn/x_68ac100d
正在爬取： https://xiamen.8684.cn/x_06534fa6
正在爬取： https://xiamen.8684.cn/x_1fdf575b
正在爬取： https://xiamen.8684.cn/x_b7b5068a
正在爬取： https://xiamen.8684.cn/x_55c1140a
正在爬取： https://xiamen.8684.cn/x_649456c8
正在爬取： https://xiamen.8684.cn/x_a0f57b23
正在爬取： https://xiamen.8684.cn/x_a0d6d78f
正在爬取： https://xiamen.8684.cn/x_6f2a7370
正在爬取： https://xiamen.8684.cn/x_b21ef2b8
正在爬取： https://xiamen.8684.cn/x_9be7f887
正在爬取： https://xiamen.8684.cn/x_d7bd9fe3
正在爬取： https://xiamen.8684.cn/x_03dec394
正在爬取： https://xiamen.8684.cn/x_a27d50e9
正在爬取： https://xiamen.8684.cn/x_d8f9fb6f
正在爬取： https://xiamen.8684.cn/x_f71b19de
正在爬取： https://xiamen.8684.cn/x_81976045
正在爬取： https://xiamen.8684.cn/x_2218ab25
正在爬取： https://xiamen.8684.cn/x_c1b9b2b4
正在爬取： https://xiamen.8684.cn/x_2ac1e226
正在爬取： https://xiamen.8684.cn/x_7ba0e147
正在爬取： https://xiamen.8684.cn/x_153e7445
正在爬取： https://xiamen.8684.cn/x_08e846af
正在爬取： https://xiamen.8684.cn/x_cb01a899
正在爬取： https://xiamen.8684.cn/x_0d615193


正在爬取： https://xiamen.8684.cn/x_7833ac5a
正在爬取： https://xiamen.8684.cn/x_1467dceb
正在爬取： https://xiamen.8684.cn/x_508724bf
正在爬取： https://xiamen.8684.cn/x_e9e6eff2
正在爬取： https://xiamen.8684.cn/x_77f23ff2
正在爬取： https://xiamen.8684.cn/x_bc572c62
正在爬取： https://xiamen.8684.cn/x_832c590b
正在爬取： https://xiamen.8684.cn/x_b26577f5
正在爬取： https://xiamen.8684.cn/x_acba7c0a
正在爬取： https://xiamen.8684.cn/x_d60df09d
正在爬取： https://xiamen.8684.cn/x_e5193f90
正在爬取： https://xiamen.8684.cn/x_fd1f2ec4
正在爬取： https://xiamen.8684.cn/x_e98e9200
正在爬取： https://xiamen.8684.cn/x_93eab4bc
正在爬取： https://xiamen.8684.cn/x_a36a6a41
正在爬取： https://xiamen.8684.cn/x_08363825
正在爬取： https://xiamen.8684.cn/x_417734f3
正在爬取： https://xiamen.8684.cn/x_b7212a31
正在爬取： https://xiamen.8684.cn/x_55f15529
正在爬取： https://xiamen.8684.cn/x_a70a7b44
正在爬取： https://xiamen.8684.cn/x_d482bab8
正在爬取： https://xiamen.8684.cn/x_a0b92382
正在爬取： https://xiamen.8684.cn/x_7e92db6c
正在爬取： https://xiamen.8684.cn/x_66101de2
正在爬取： https://xiamen.8684.cn/x_91666e73


In [6]:
for i in out_url:
    get_info2(i)

正在爬取： https://xiamen.8684.cn/x_eb7745fd
正在爬取： https://xiamen.8684.cn/x_a4064afc
正在爬取： https://xiamen.8684.cn/x_bfe4a3eb
正在爬取： https://xiamen.8684.cn/x_7db14486
正在爬取： https://xiamen.8684.cn/x_4afa9e0d
正在爬取： https://xiamen.8684.cn/x_32d0d5d7
正在爬取： https://xiamen.8684.cn/x_a6a52ff7
正在爬取： https://xiamen.8684.cn/x_90e2ca26
正在爬取： https://xiamen.8684.cn/x_3ee317fa
正在爬取： https://xiamen.8684.cn/x_fec27493
正在爬取： https://xiamen.8684.cn/x_a3fc848f
正在爬取： https://xiamen.8684.cn/x_4542b81e
正在爬取： https://xiamen.8684.cn/x_92abb355
正在爬取： https://xiamen.8684.cn/x_dad74d6b
正在爬取： https://xiamen.8684.cn/x_a14b696b
正在爬取： https://xiamen.8684.cn/x_497aa162
正在爬取： https://xiamen.8684.cn/x_450c98e4
正在爬取： https://xiamen.8684.cn/x_bb5dd285
正在爬取： https://xiamen.8684.cn/x_7f2bbd44
正在爬取： https://xiamen.8684.cn/x_4f82091d
正在爬取： https://xiamen.8684.cn/x_36fc37ab
正在爬取： https://xiamen.8684.cn/x_ac5cf874
正在爬取： https://xiamen.8684.cn/x_5b9593cd
正在爬取： https://xiamen.8684.cn/x_6674c2bb
正在爬取： https://xiamen.8684.cn/x_e870d975


正在爬取： https://xiamen.8684.cn/x_68ac100d
正在爬取： https://xiamen.8684.cn/x_06534fa6
正在爬取： https://xiamen.8684.cn/x_1fdf575b
正在爬取： https://xiamen.8684.cn/x_b7b5068a
正在爬取： https://xiamen.8684.cn/x_55c1140a
正在爬取： https://xiamen.8684.cn/x_649456c8
正在爬取： https://xiamen.8684.cn/x_a0f57b23
正在爬取： https://xiamen.8684.cn/x_a0d6d78f
正在爬取： https://xiamen.8684.cn/x_6f2a7370
正在爬取： https://xiamen.8684.cn/x_b21ef2b8
正在爬取： https://xiamen.8684.cn/x_9be7f887
正在爬取： https://xiamen.8684.cn/x_d7bd9fe3
正在爬取： https://xiamen.8684.cn/x_03dec394
正在爬取： https://xiamen.8684.cn/x_a27d50e9
正在爬取： https://xiamen.8684.cn/x_d8f9fb6f
正在爬取： https://xiamen.8684.cn/x_f71b19de
正在爬取： https://xiamen.8684.cn/x_81976045
正在爬取： https://xiamen.8684.cn/x_2218ab25
正在爬取： https://xiamen.8684.cn/x_c1b9b2b4
正在爬取： https://xiamen.8684.cn/x_2ac1e226
正在爬取： https://xiamen.8684.cn/x_7ba0e147
正在爬取： https://xiamen.8684.cn/x_153e7445
正在爬取： https://xiamen.8684.cn/x_08e846af
正在爬取： https://xiamen.8684.cn/x_cb01a899
正在爬取： https://xiamen.8684.cn/x_0d615193


正在爬取： https://xiamen.8684.cn/x_7833ac5a
正在爬取： https://xiamen.8684.cn/x_1467dceb
正在爬取： https://xiamen.8684.cn/x_508724bf
正在爬取： https://xiamen.8684.cn/x_e9e6eff2
正在爬取： https://xiamen.8684.cn/x_77f23ff2
正在爬取： https://xiamen.8684.cn/x_bc572c62
正在爬取： https://xiamen.8684.cn/x_832c590b
正在爬取： https://xiamen.8684.cn/x_b26577f5
正在爬取： https://xiamen.8684.cn/x_acba7c0a
正在爬取： https://xiamen.8684.cn/x_d60df09d
正在爬取： https://xiamen.8684.cn/x_e5193f90
正在爬取： https://xiamen.8684.cn/x_fd1f2ec4
正在爬取： https://xiamen.8684.cn/x_e98e9200
正在爬取： https://xiamen.8684.cn/x_93eab4bc
正在爬取： https://xiamen.8684.cn/x_a36a6a41
正在爬取： https://xiamen.8684.cn/x_08363825
正在爬取： https://xiamen.8684.cn/x_417734f3
正在爬取： https://xiamen.8684.cn/x_b7212a31
正在爬取： https://xiamen.8684.cn/x_55f15529
正在爬取： https://xiamen.8684.cn/x_a70a7b44
正在爬取： https://xiamen.8684.cn/x_d482bab8
正在爬取： https://xiamen.8684.cn/x_a0b92382
正在爬取： https://xiamen.8684.cn/x_7e92db6c
正在爬取： https://xiamen.8684.cn/x_66101de2
正在爬取： https://xiamen.8684.cn/x_91666e73


In [13]:
df1.to_excel('xm_bus_info1.xls')
df2.to_excel('xm_bus_info2.xls')

C:\Users\12461\AppData\Local\Temp\ipykernel_11028\2559844844.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df1.to_excel('xm_bus_info1.xls')
C:\Users\12461\AppData\Local\Temp\ipykernel_11028\2559844844.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning

In [7]:
df1

,路线,方向,运行时间,参考票价,公交公司,线路类型,途径站点,备注
0,厦门912路公交车路线,软件园三期-东安,软件园三期 06:20-20:10,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团集美公交公司,[岛外常规公交线路],"[软件园三期, 软件园三期起步区, 软件园三期龙亭, 莲花新城, 龙亭三里北, 诚毅南路, ...",None
1,厦门912路公交车路线,东安-软件园三期,东安 06:30-20:20,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团集美公交公司,[岛外常规公交线路],"[东安, 东安后垵, 美山小学, 东安路, 集美中学高中部, 霞梧, 侨英小学, 侨英路口,...",None
2,厦门M26路公交车路线,新阳公交场站-地铁东瑶站,新阳公交场站 06:10-23:40,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团海沧公交公司,[岛外常规公交线路],"[新阳公交场站, 海沧中学, 地铁新阳大道站, 新景西二路口, 后柯村口, 温头溪, 一中海...",None
3,厦门M26路公交车路线,地铁东瑶站-新阳公交场站,地铁东瑶站 06:10-23:45,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团海沧公交公司,[岛外常规公交线路],"[地铁东瑶站, 一中海沧校区东门, 一中海沧校区, 温头溪, 后柯村口, 新景西二路口, 地...",None
4,厦门18路公交车路线,黄厝公交场站-园山,黄厝公交场站 05:55-22:00,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[黄厝公交场站, 黄厝海滨, 黄厝浴场, 椰风寨, 黄厝石胄头, 环岛东路, 会展南五路, ...",None
...,...,...,...,...,...,...,...,...
817,厦门10路公交车路线,轮渡公交场站-乌石浦油画村总站,轮渡公交场站 05:45-22:50,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[轮渡公交场站, 开禾路口, 思北路口, 角滨路口, 十一中, 滨南美湖路口, 非矿, 滨南...",None
818,厦门750路[快线]公交车路线,马巷农客站-梧村公交场站,公交车路线[跨区常规公交线路]马巷农客站 05:10-21:30,常规公交：一票制2元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团翔安公交公司,[跨区常规公交线路],"[马巷农客站, 翔安公路分局, 马巷车站, 第一医院同民分院, 下坂, 西亭, 火炬路口, ...",None
819,厦门750路[快线]公交车路线,梧村公交场站-马巷农客站,梧村公交场站 06:20-23:00,常规公交：一票制2元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团翔安公交公司,[跨区常规公交线路],"[梧村公交场站, 吕厝北, 江头花园, 中医院, 国贸中心, 翔安肖厝, 林前, 海洋学院翔...",None
820,厦门3路公交车路线,莲花五村-第一码头,莲花五村 05:50-23:00,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[莲花五村, 龙华里, 龙昌路口, 龙盛里, 龙昌里, 泰和花园, 莲花三村, 莲花二村, ...",None


In [8]:
df2

,路线,方向,运行时间,参考票价,公交公司,线路类型,途径站点,备注
0,厦门912路公交车路线,软件园三期-东安,软件园三期 06:20-20:10,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团集美公交公司,[岛外常规公交线路],"[软件园三期, 软件园三期起步区, 软件园三期龙亭, 莲花新城, 龙亭三里北, 诚毅南路, ...",None
1,厦门912路公交车路线,东安-软件园三期,东安 06:30-20:20,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团集美公交公司,[岛外常规公交线路],"[东安, 东安后垵, 美山小学, 东安路, 集美中学高中部, 霞梧, 侨英小学, 侨英路口,...",None
2,厦门M26路公交车路线,新阳公交场站-地铁东瑶站,新阳公交场站 06:10-23:40,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团海沧公交公司,[岛外常规公交线路],"[新阳公交场站, 海沧中学, 地铁新阳大道站, 新景西二路口, 后柯村口, 温头溪, 一中海...",None
3,厦门M26路公交车路线,地铁东瑶站-新阳公交场站,地铁东瑶站 06:10-23:45,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团海沧公交公司,[岛外常规公交线路],"[地铁东瑶站, 一中海沧校区东门, 一中海沧校区, 温头溪, 后柯村口, 新景西二路口, 地...",None
4,厦门18路公交车路线,黄厝公交场站-园山,黄厝公交场站 05:55-22:00,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[黄厝公交场站, 黄厝海滨, 黄厝浴场, 椰风寨, 黄厝石胄头, 环岛东路, 会展南五路, ...",None
...,...,...,...,...,...,...,...,...
832,厦门10路公交车路线,轮渡公交场站-乌石浦油画村总站,轮渡公交场站 05:45-22:50,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[轮渡公交场站, 开禾路口, 思北路口, 角滨路口, 十一中, 滨南美湖路口, 非矿, 滨南...",None
833,厦门750路[快线]公交车路线,马巷农客站-梧村公交场站,公交车路线[跨区常规公交线路]马巷农客站 05:10-21:30,常规公交：一票制2元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团翔安公交公司,[跨区常规公交线路],"[马巷农客站, 翔安公路分局, 马巷车站, 第一医院同民分院, 下坂, 西亭, 火炬路口, ...",None
834,厦门750路[快线]公交车路线,梧村公交场站-马巷农客站,梧村公交场站 06:20-23:00,常规公交：一票制2元，E通卡8折，支持移动支付，支持换乘优惠,厦门公交集团翔安公交公司,[跨区常规公交线路],"[梧村公交场站, 吕厝北, 江头花园, 中医院, 国贸中心, 翔安肖厝, 林前, 海洋学院翔...",None
835,厦门3路公交车路线,莲花五村-第一码头,莲花五村 05:50-23:00,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛内常规公交线路],"[莲花五村, 龙华里, 龙昌路口, 龙盛里, 龙昌里, 泰和花园, 莲花三村, 莲花二村, ...",None


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822 entries, 0 to 821
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   路线      822 non-null    object
 1   方向      822 non-null    object
 2   运行时间    822 non-null    object
 3   参考票价    822 non-null    object
 4   公交公司    822 non-null    object
 5   线路类型    822 non-null    object
 6   途径站点    822 non-null    object
 7   备注      313 non-null    object
dtypes: object(8)
memory usage: 57.8+ KB


In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 837 entries, 0 to 836
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   路线      837 non-null    object
 1   方向      837 non-null    object
 2   运行时间    837 non-null    object
 3   参考票价    837 non-null    object
 4   公交公司    837 non-null    object
 5   线路类型    811 non-null    object
 6   途径站点    837 non-null    object
 7   备注      328 non-null    object
dtypes: object(8)
memory usage: 58.9+ KB


In [14]:
df1.describe()

,路线,方向,运行时间,参考票价,公交公司,线路类型,途径站点,备注
count,822,822,822,822,822,822,822,313
unique,426,799,801,24,15,9,821,133
top,厦门912路公交车路线,文屏-七星公交场站,公交车路线[岛内常规公交线路]详见备注,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛外常规公交线路],"[乐海公交场站, 凤岗, 柑岭, 刘营, 湖柑大安, 湖柑, 湖井, 内辽, 白沙仑, 莲花...",【工作日运营】
freq,2,3,6,628,272,340,2,29


In [15]:
df2.describe()

,路线,方向,运行时间,参考票价,公交公司,线路类型,途径站点,备注
count,837,837,837,837,837,811,837,328
unique,434,810,815,29,16,9,836,136
top,厦门912路公交车路线,七星公交场站-文屏,公交车路线[岛内常规公交线路]详见备注,常规公交：一票制1元，E通卡8折，支持移动支付，支持换乘优惠,厦门鹭城巴士集团有限公司,[岛外常规公交线路],"[乐海公交场站, 凤岗, 柑岭, 刘营, 湖柑大安, 湖柑, 湖井, 内辽, 白沙仑, 莲花...",【工作日运营】
freq,2,3,6,623,289,337,2,29
